# Lecture 2 - text data

Language is the tool we use to communicate, think, store knowledge. Its the key! 

Text data is a very hard data type to work with : 

- High Dimensionality: Text data, especially when tokenized into individual words or n-grams, can lead to a very high-dimensional feature space. This can make models computationally expensive and increase the risk of overfitting.

- Sparsity: Most text documents will only contain a tiny fraction of the words in a language, resulting in many zeros in the feature representation. This sparsity can make certain modeling techniques inefficient or infeasible.

- Ambiguity and Polysemy: Many words in languages have multiple meanings based on context. For example, the word "bank" can mean the side of a river or a financial institution.

- Synonymy: Different words can have similar meanings, like "big" and "large". This poses a challenge in identifying the true intent or sentiment behind texts.

- Complex Dependencies: The meaning of a word can depend on its surrounding words, or even words much earlier in a text. Capturing long-term dependencies can be challenging.

- Variability: Texts can express the same idea in myriad ways due to differences in style, tone, or choice of vocabulary.

- Noisy Data: Text data, especially from sources like social media, can be noisy. They may contain typos, slang, non-standard grammar, or emoticons.

- Handling of Out-of-Vocabulary (OOV) Words: In real-world applications, it's common to encounter words not seen during training. Handling OOV words is challenging.

- Cultural and Temporal Dynamics: The way language is used can change based on cultural or temporal contexts, making it hard to generalize models across different cultures or time periods.

- Data Labeling Challenges: For supervised learning, text data may require manual labeling. This can be time-consuming, expensive, and subjective. Inter-annotator disagreement can also be an issue.

- Bias and Fairness: Text data, especially from the web, can contain biases. Machine learning models trained on such data can inherit and amplify these biases, leading to unfair or prejudiced predictions.

- Scalability: Processing and analyzing large text corpora require significant computational resources and efficient algorithms.

- Multilingual Challenges: Building models that work across multiple languages or even dialects can be challenging, especially when resources for certain languages are scarce.

In [1]:
categories = ['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med']

In [2]:
from sklearn.datasets import fetch_20newsgroups

In [3]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

In [4]:
len(twenty_train.data)

2257